# Tarea 1 - Ciencia de Datos
### Por: Gustavo Hernández Angeles

## Preparación

In [ ]:
import numpy as np
from ucimlrepo import fetch_ucirepo 

{'uci_id': 53, 'name': 'Iris', 'repository_url': 'https://archive.ics.uci.edu/dataset/53/iris', 'data_url': 'https://archive.ics.uci.edu/static/public/53/data.csv', 'abstract': 'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 150, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1936, 'last_updated': 'Tue Sep 12 2023', 'dataset_doi': '10.24432/C56C76', 'creators': ['R. A. Fisher'], 'intro_paper': {'ID': 191, 'type': 'NATIVE', 'title': 'The Iris data set: In search of the source of virginica', 'authors': 'A. Unwin, K. Kleinman', 'venue': 'Significance, 2021', 'year': 2021, 'journal': 'Significance, 2021', 'DOI': '1740-9713.01589', 'URL': 'https://www.semanticscholar.org

In [ ]:
# Definimos la función de distancia
def distancia_minkowski(x, y, n):
    return np.sum(np.abs(x-y)**n)**(1/n)

# Definimos la función de Clustering Jerárquico Aglomerativo
def jerarquico_aglomerativo(X, n_clusters):
    n = X.shape[0]
    # Cada punto es un cluster
    clusters = [[i] for i in range(n)]
    # Inicializar matriz de disimilitud
    matriz_disim = np.array([[distancia_minkowski(X[i],X[j]) for j in range(n)] for i in range(n)])
    
    # Hacer hasta obtener n_clusters clusters
    for t in range(n-n_clusters):